In [2]:
import pandas as pd

# Fuzzy Matching - 04082020

In [21]:
old=pd.read_excel(r'data\2015 Booth List.xlsx')
new=pd.read_excel(r'data\2019 Booth List.xlsx')

In [6]:
# I. Fuzzy Matching the intra localities - both old & new

In [163]:
old.head(5)

,booth_number,booth_name
0,1,Uchcha Vighalay Jamhor Purwi Bhag
1,2,Uchcha Vighalay Jamhor Purwi Madhya Bhag
2,3,Uchcha Vighalay Jamhor Pashchimi Bhag
3,4,Uchcha Vighalay Jamhor Pashchimi Madhya Bhag
4,5,Madhya Vighalay Jamhor Purwi Bhag


In [7]:
unique_loc=new['booth_name'].unique().tolist()
type(unique_loc)

list

In [4]:
from fuzzywuzzy import process, fuzz

C:\anaconda\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# LOCALITY FUZZY - edit 10082020 - (dont use, do direct match on booth first

In [8]:
# Intra fuzzy for NEW
score_sort = [(x,) + i
             for x in unique_loc 
             for i in process.extract(x, unique_loc,     scorer=fuzz.token_sort_ratio)]
#Create a dataframe from the tuples
similarity_sort = pd.DataFrame(score_sort, columns=['loc_sort','match_sort','score_sort'])
similarity_sort.head()

#Derive representative values
import numpy as np
similarity_sort['sorted_loc_sort'] = np.minimum(similarity_sort['loc_sort'], similarity_sort['match_sort'])
similarity_sort.head()

high_score_sort = similarity_sort[(similarity_sort['score_sort'] >= 90) &
                                      (similarity_sort['loc_sort'] != similarity_sort['match_sort']) &
                                      (similarity_sort['sorted_loc_sort'] != similarity_sort['match_sort'])]

new1=new.merge(high_score_sort,how='left',left_on=new['locality'].str.upper().str.strip(),\
         right_on=high_score_sort['match_sort'].str.upper().str.strip(),indicator=True)

new1['loc_sort1']=np.where(new1['_merge']=='left_only',new1['locality'],new1['sorted_loc_sort'])

# Intra Fuzzy for OLD

unique_loc1=old['locality'].unique().tolist()
score_sort = [(x,) + i
             for x in unique_loc 
             for i in process.extract(x, unique_loc,     scorer=fuzz.token_sort_ratio)]


#Create a dataframe from the tuples
similarity_sort = pd.DataFrame(score_sort, columns=['loc_sort','match_sort','score_sort'])
similarity_sort.head()

#Derive representative values
import numpy as np
similarity_sort['sorted_loc_sort'] = np.minimum(similarity_sort['loc_sort'], similarity_sort['match_sort'])
similarity_sort.head()

high_score_sort = similarity_sort[(similarity_sort['score_sort'] >= 90) &
                                      (similarity_sort['loc_sort'] != similarity_sort['match_sort']) &
                                      (similarity_sort['sorted_loc_sort'] != similarity_sort['match_sort'])]

old1=old.merge(high_score_sort,how='left',left_on=old['locality'].str.upper().str.strip(),\
         right_on=high_score_sort['match_sort'].str.upper().str.strip(),indicator=True)

old1['loc_sort1']=np.where(old1['_merge']=='left_only',old1['locality'],old1['sorted_loc_sort'])

In [78]:
old1[old1['loc_sort1']=='Ambedkar Aawasiya High School']

,key_0,Number,Name,locality,loc_sort,match_sort,score_sort,sorted_loc_sort,_merge,loc_sort1
211,AMBEDKAR AAWASIYA HIGH SCHOOL,212,Ambedkar Aawasiya High School Kishanganj East ...,Ambedkar Aawasiya High School,NaN,NaN,NaN,NaN,left_only,Ambedkar Aawasiya High School
212,AMBEDKAR AAWASIYA HIGH SCHOOL,213,Ambedkar Aawasiya High School Kishanganj Middl...,Ambedkar Aawasiya High School,NaN,NaN,NaN,NaN,left_only,Ambedkar Aawasiya High School
213,AMBEDKAR AAWASIYA HIGH SCHOOL,214,Ambedkar Aawasiya High School Kishanganj West ...,Ambedkar Aawasiya High School,NaN,NaN,NaN,NaN,left_only,Ambedkar Aawasiya High School


In [80]:
old1[old1['loc_sort1']=='pahadakatata']

,key_0,Number,Name,locality,loc_sort,match_sort,score_sort,sorted_loc_sort,_merge,loc_sort1
128,PAHADAKATTA,129,"Panchayat Bhawan Pahadakatta, East Part",pahadakatta,pahadakatata,pahadakatta,96.0,pahadakatata,both,pahadakatata
129,PAHADAKATTA,130,Panchayat Bhawan Pahadakatta West Part,pahadakatta,pahadakatata,pahadakatta,96.0,pahadakatata,both,pahadakatata


# BOOTH FUZZY - USE THIS

In [22]:
# Inter Fuzzy between new1 and old1


def checker(wrong_options,correct_options):
    
    first_pref=[]
    names_array=[]
    ratio_array=[]
    second_pref=[]
    third_pref=[]
    fourth_pref=[]
    fifth_pref=[]
    for wrong_option in wrong_options:
        
        x=process.extractBests(wrong_option,correct_options,scorer=fuzz.token_sort_ratio)
        first_pref.append(x[0])
        second_pref.append(x[1])
        third_pref.append(x[2])
        fourth_pref.append(x[3])
        fifth_pref.append(x[4])
    return first_pref,second_pref,third_pref,fourth_pref,fifth_pref

In [23]:
old_names=old['booth_name'].tolist()
new_names=new['booth_name'].tolist()
new_booth=new['booth_number'].tolist()

In [24]:
len(new_names),new.shape

(322, (322, 2))

In [25]:
first_pref,second_pref,third_pref,fourth_pref,fifth_pref=checker(new_names,old_names)


df1 = pd.DataFrame()
df1['new_names']=pd.Series(new_names)
df1['booth_num']=pd.Series(new_booth)
# df1['perfect_match']=pd.Series(names_array)
# df1['perfect_ratio']=pd.Series(ratio_array)
df1['first_pref']=pd.Series(i[0] for i in first_pref)
df1['first_score']=pd.Series(i[1] for i in first_pref)
df1['second_pref']=pd.Series(i[0] for i in second_pref)
df1['second_score']=pd.Series(i[1] for i in second_pref)
df1['third_pref']=pd.Series(i[0] for i in third_pref)
df1['third_score']=pd.Series(i[1] for i in third_pref)
df1['fourth_pref']=pd.Series(i[0] for i in fourth_pref)
df1['fourth_score']=pd.Series(i[1] for i in fourth_pref)
df1['fifth_pref']=pd.Series(i[0] for i in fifth_pref)
df1['fifth_score']=pd.Series(i[1] for i in fifth_pref)
# df1.to_excel(‘matched_names.xlsx’, engine=’xlsxwriter’)

In [27]:
df1.shape,len(new_names)

((322, 12), 322)

In [28]:
pd.set_option('display.max_colwidth', -1)

In [29]:
df1[df1['first_score']==100].shape,df1.shape

((112, 12), (322, 12))

In [32]:
booth_match=df1[df1['first_score']>=95]

In [33]:
booth_match['match_type']='booth'

C:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [204]:
# This contains all booth matches - 140 booths
booth_match.shape

# remaining to be matched using locality - to be matched 182 booths
local_match=df1.loc[:,['new_names','booth_num','first_score']]



In [205]:
df1.shape

(322, 12)

In [206]:
local_match.shape,booth_match.shape,new.shape

((322, 3), (140, 13), (322, 2))

In [258]:
df1.to_excel(r'matched_names_new1.xlsx',index=False)

# Locality Fuzzy 

In [207]:
#1 removing alpha numeric
import re

local_match['name1']=local_match['new_names'].apply(lambda x : re.sub(r'[^a-zA-Z0-9]',' ',x).upper())
old['name1']=old['booth_name'].apply(lambda x : re.sub(r'[^a-zA-Z0-9]',' ',x).upper())

# Overall Locality fuzzy for df1


In [208]:
# 2 removing common words
to_remove=["west","part","north","south","room"," no ",\
                     "kishanganj","middle","east","office",\
                     "building","hat","new","madarsa","taluka",\
                     "dakshin","bhawan"," nagar",\
                     "purab","purana","Bhag","VIGHALAY","PURWI",'PRAMANDALA',"MADHYA","UCHCHA",'BAS PADAV','VIGHALAY','SABHAGAR','BHAG','PASHCHIMI','VIDHAYAL',\
          'VIDYALAY','UTKRAMIT','SAMUDAYIK','PRATHAMIK','PRATHMIK','PURVI','URDU','MAKHATAB','MADYA','PASCHIMI','BHAVAN',\
          'UCHUCH','VIDHYALAY','PASHCHIM','DISHA','PAKSHCHIM','MADHY','BHAWAN','KHURD','VIDYALAY','UTTARI','UTARI','UTAR','DAKSHINI',\
          'SASKRIT','HARIJAN','MANDIR','RAJARSHIVIDYA','RAJARSHI','PURABI','SAAR','DAKSHIN','KANYA',\
           'LOK','SWASTHYA','PRAMANDAL','PRAMANDALA','KARYALAY','SINHA','SHOSHAL','KALAB','ANUGRAH',\
           'INTAR','KOLEJA','PURWI','VIGYAN','PAHALA','KAMARA','KAMARA NO 2 DAYAN','KAMRA NO 02 BAYAN','MADARSHA','JILA',\
           'PARISHAD','KAMARA','NO','3','4','5','SASKRIT','PASHUPALAN','UTTAR','MADARASA','ISLAMIYA','MADARSHA',\
           'ROD','ROAD','BAPU','SACHCHIDANANDA','SINHA','KOLEJ','MINI','ISLAM','PATHAN','MAHALLA','DAKSHIN',\
           'TAUN','INTAR','KOLEJ','ABHIYANTRAN','VI KA','KAVARTAR','MAHALLA','TICHAR','TRAAINING','SCHOOL','RUMANO 1','RUMANO2',\
           'RUMA','MALLAH','RAMALAKHAN SINH YADAV KOLEJ','BESIK','SCHOOL',\
           'SKUL','2','1','BAYAN','PANCHAYAT','VIDHALAY','BIGAHA','BIGHA','VIDHALYA','PAKSHICHAM','PURW','NA0','NAO','0',\
          'KAMRA','DAYAN','VIDYA','SAAR','KALBA']


# Function1
# def remove_words(curr_string,replace_list):
#     for curr_word in replace_list:
#         curr_string=curr_string.replace(curr_word,'')
#     return curr_string

# using regex

                              
local_match['name2']=local_match['name1'].apply(lambda x : re.sub(r'|'.join(map(re.escape, to_remove)), '', x)).apply(lambda x : x.strip())  
old['name2']=old['name1'].apply(lambda x : re.sub(r'|'.join(map(re.escape, to_remove)), '', x)).apply(lambda x : x.strip())


# some booths do not have locality names in them - renaming locality for them as 'others' 
local_match['name2']=np.where(local_match['name2']=='','Others',local_match['name2'])
old['name2']=np.where(old['name2']=='','Others',old['name2'])


In [209]:
# 3 stripping spaces and taking last word as locality
def lword(x):
    return x.strip().split()[-1]

local_match['name2']=local_match['name2'].apply(lambda x : lword(x))
old['name2']=old['name2'].apply(lambda x : lword(x))

In [200]:
local_match.head()

,new_names,booth_num,name1,name2
0,Uchcha Vighalay Jamhor Purwi Bhag.,1,UCHCHA VIGHALAY JAMHOR PURWI BHAG,JAMHOR
1,Uchcha Vighalay Jamhor Purwi Madhya Bhag,2,UCHCHA VIGHALAY JAMHOR PURWI MADHYA BHAG,JAMHOR
2,Uchcha Vighalay Jamhor Pashchimi Madhya Bhag,3,UCHCHA VIGHALAY JAMHOR PASHCHIMI MADHYA BHAG,JAMHOR
3,Uchcha Vighalay Jamhor Pashchimi Madhya Bhag,4,UCHCHA VIGHALAY JAMHOR PASHCHIMI MADHYA BHAG,JAMHOR
4,Madhya Vighalay Jamhor Purwi Bhag,5,MADHYA VIGHALAY JAMHOR PURWI BHAG,JAMHOR


In [210]:
old_local=old['name2'].unique().tolist()
new_local=local_match['name2'].unique().tolist()
len(new_local),len(old_local)

(193, 182)

In [211]:
# 4 Intra fuzzy for NEW
score_sort = [(x,) + i
             for x in new_local 
             for i in process.extract(x, new_local,     scorer=fuzz.token_sort_ratio)]
#Create a dataframe from the tuples
similarity_sort = pd.DataFrame(score_sort, columns=['loc_sort','match_sort','score_sort'])
similarity_sort.head()

import numpy as np
similarity_sort['sorted_loc_sort'] = np.minimum(similarity_sort['loc_sort'], similarity_sort['match_sort'])
similarity_sort.head()

#Derive representative values


high_score_sort = similarity_sort[(similarity_sort['score_sort'] >= 90) &
                                      (similarity_sort['loc_sort'] != similarity_sort['match_sort']) &
                                      (similarity_sort['sorted_loc_sort'] != similarity_sort['match_sort'])]

local_match1=local_match.merge(high_score_sort,how='left',left_on=local_match['name2'].str.upper().str.strip(),\
         right_on=high_score_sort['match_sort'].str.upper().str.strip(),indicator=True)

local_match1['loc_sort1']=np.where(local_match1['_merge']=='left_only',local_match1['name2'],local_match1['sorted_loc_sort'])

In [212]:
local_match1.head(1)

,key_0,new_names,booth_num,first_score,name1,name2,loc_sort,match_sort,score_sort,sorted_loc_sort,_merge,loc_sort1
0,JAMHOR,Uchcha Vighalay Jamhor Purwi Bhag.,1,100,UCHCHA VIGHALAY JAMHOR PURWI BHAG,JAMHOR,NaN,NaN,NaN,NaN,left_only,JAMHOR


In [213]:
# 5 Intra Fuzzy for OLD

score_sort = [(x,) + i
             for x in old_local 
             for i in process.extract(x, old_local,     scorer=fuzz.token_sort_ratio)]


#Create a dataframe from the tuples
similarity_sort = pd.DataFrame(score_sort, columns=['loc_sort','match_sort','score_sort'])
similarity_sort.head()

#Derive representative values
import numpy as np
similarity_sort['sorted_loc_sort'] = np.minimum(similarity_sort['loc_sort'], similarity_sort['match_sort'])
similarity_sort.head()

high_score_sort = similarity_sort[(similarity_sort['score_sort'] >= 90) &
                                      (similarity_sort['loc_sort'] != similarity_sort['match_sort']) &
                                      (similarity_sort['sorted_loc_sort'] != similarity_sort['match_sort'])]

old1=old.merge(high_score_sort,how='left',left_on=old['name2'].str.upper().str.strip(),\
         right_on=high_score_sort['match_sort'].str.upper().str.strip(),indicator=True)

old1['loc_sort1']=np.where(old1['_merge']=='left_only',old1['name2'],old1['sorted_loc_sort'])

In [214]:
local_match1.head()

,key_0,new_names,booth_num,first_score,name1,name2,loc_sort,match_sort,score_sort,sorted_loc_sort,_merge,loc_sort1
0,JAMHOR,Uchcha Vighalay Jamhor Purwi Bhag.,1,100,UCHCHA VIGHALAY JAMHOR PURWI BHAG,JAMHOR,NaN,NaN,NaN,NaN,left_only,JAMHOR
1,JAMHOR,Uchcha Vighalay Jamhor Purwi Madhya Bhag,2,100,UCHCHA VIGHALAY JAMHOR PURWI MADHYA BHAG,JAMHOR,NaN,NaN,NaN,NaN,left_only,JAMHOR
2,JAMHOR,Uchcha Vighalay Jamhor Pashchimi Madhya Bhag,3,100,UCHCHA VIGHALAY JAMHOR PASHCHIMI MADHYA BHAG,JAMHOR,NaN,NaN,NaN,NaN,left_only,JAMHOR
3,JAMHOR,Uchcha Vighalay Jamhor Pashchimi Madhya Bhag,4,100,UCHCHA VIGHALAY JAMHOR PASHCHIMI MADHYA BHAG,JAMHOR,NaN,NaN,NaN,NaN,left_only,JAMHOR
4,JAMHOR,Madhya Vighalay Jamhor Purwi Bhag,5,100,MADHYA VIGHALAY JAMHOR PURWI BHAG,JAMHOR,NaN,NaN,NaN,NaN,left_only,JAMHOR


In [215]:
# 6 inter fuzzy between localilities from local_match1(only for those who have first_Score<95) & old1

new_local=local_match1.loc[local_match1['first_score']<95,'loc_sort1'].unique().tolist()
old_local=old1['loc_sort1'].unique().tolist()

In [217]:
len(new_local),len(old_local)

(85, 179)

In [218]:
first_pref,second_pref,third_pref,fourth_pref,fifth_pref=checker(new_local,old_local)


df3 = pd.DataFrame()
df3['new_local']=pd.Series(new_local)
# df3['booth_num']=pd.Series(new_booth)
# df3['perfect_match']=pd.Series(names_array)
# df3['perfect_ratio']=pd.Series(ratio_array)
df3['first_pref']=pd.Series(i[0] for i in first_pref)
df3['first_score']=pd.Series(i[1] for i in first_pref)
df3['second_pref']=pd.Series(i[0] for i in second_pref)
df3['second_score']=pd.Series(i[1] for i in second_pref)
df3['third_pref']=pd.Series(i[0] for i in third_pref)
df3['third_score']=pd.Series(i[1] for i in third_pref)
df3['fourth_pref']=pd.Series(i[0] for i in fourth_pref)
df3['fourth_score']=pd.Series(i[1] for i in fourth_pref)
df3['fifth_pref']=pd.Series(i[0] for i in fifth_pref)
df3['fifth_score']=pd.Series(i[1] for i in fifth_pref)
# df3.to_excel(‘matched_names.xlsx’, engine=’xlsxwriter’)

In [219]:
df3[df3['first_score']>=85].shape,df3.shape

((70, 11), (85, 11))

In [175]:
df3

,new_names,first_pref,first_score,second_pref,second_score,third_pref,third_score,fourth_pref,fourth_score,fifth_pref,fifth_score
0,JAMHOR,JAMHOR,100,GAMHARI,62,JOKAHARI,57,BAJAR,55,JARAMAKHAP,50
1,RAJA,RAJA,100,AJAB,75,KARANJA,73,RAJUPAR,73,RAJOI,67
2,DIHRI,DIHARI,91,DHANARI,67,DEVADHARI,57,KHAIRI,55,DUMARI,55
3,PADRAWAN,PADARAVA,75,PARASA,71,PAHARAMA,62,PADARIYA,62,POKHARAHA,59
4,DEVARIYA,DEVARIYA,100,DEVAHARA,75,TETARIYA,75,PADARIYA,75,DARIYAPUR,71
5,CHITRAGOPI,CHITRAGOPI,100,CHIRIYA,59,CHATARA,59,BAGOI,53,RAJOI,53
6,BHARATHAULI,BHARATHAULI,100,BARATAPUR,70,RADHAULIYA,67,HASAULI,67,HARANAHI,63
7,BATAVA,BATAVA,100,BADAVA,83,SATAVAT,77,PATANAVA,71,BHARAVAR,71
8,RAMPUR,RAMAPUR,92,RAYAPURA,71,IBRAHIMAPUR,71,ALAMAPUR,71,VISHRAMAPUR,71
9,KATHAUTIRYA,KATHAUTIYA,95,KATAIYA,78,KHAIRA,71,RADHAULIYA,67,KAPASIYA,63


In [220]:
# 7 matching the booth localities in local_match1 with a df3

local_match1.head()

,key_0,new_names,booth_num,first_score,name1,name2,loc_sort,match_sort,score_sort,sorted_loc_sort,_merge,loc_sort1
0,JAMHOR,Uchcha Vighalay Jamhor Purwi Bhag.,1,100,UCHCHA VIGHALAY JAMHOR PURWI BHAG,JAMHOR,NaN,NaN,NaN,NaN,left_only,JAMHOR
1,JAMHOR,Uchcha Vighalay Jamhor Purwi Madhya Bhag,2,100,UCHCHA VIGHALAY JAMHOR PURWI MADHYA BHAG,JAMHOR,NaN,NaN,NaN,NaN,left_only,JAMHOR
2,JAMHOR,Uchcha Vighalay Jamhor Pashchimi Madhya Bhag,3,100,UCHCHA VIGHALAY JAMHOR PASHCHIMI MADHYA BHAG,JAMHOR,NaN,NaN,NaN,NaN,left_only,JAMHOR
3,JAMHOR,Uchcha Vighalay Jamhor Pashchimi Madhya Bhag,4,100,UCHCHA VIGHALAY JAMHOR PASHCHIMI MADHYA BHAG,JAMHOR,NaN,NaN,NaN,NaN,left_only,JAMHOR
4,JAMHOR,Madhya Vighalay Jamhor Purwi Bhag,5,100,MADHYA VIGHALAY JAMHOR PURWI BHAG,JAMHOR,NaN,NaN,NaN,NaN,left_only,JAMHOR


In [221]:
matched=local_match1.loc[local_match1['first_score']<95,['new_names','booth_num','loc_sort1']].merge(df3,how='left',left_on='loc_sort1',right_on='new_local',indicator='no locality match')
matched['match_type']='locality'

In [222]:
matched.shape,local_match.shape

((182, 16), (322, 5))

In [223]:
matched[matched['no locality match']=='left_only']

,new_names,booth_num,loc_sort1,new_local,first_pref,first_score,second_pref,second_score,third_pref,third_score,fourth_pref,fourth_score,fifth_pref,fifth_score,no locality match,match_type


In [226]:
local_match1.head(1)

,key_0,new_names,booth_num,first_score,name1,name2,loc_sort,match_sort,score_sort,sorted_loc_sort,_merge,loc_sort1
0,JAMHOR,Uchcha Vighalay Jamhor Purwi Bhag.,1,100,UCHCHA VIGHALAY JAMHOR PURWI BHAG,JAMHOR,NaN,NaN,NaN,NaN,left_only,JAMHOR


In [228]:
# concat matched (182 booths) and booth_match(140 booths)

booth_match1=booth_match.merge(local_match1[['new_names','booth_num','loc_sort1']],how='left',on='booth_num')

In [229]:
booth_match1

,new_names_x,booth_num,first_pref,first_score,second_pref,second_score,third_pref,third_score,fourth_pref,fourth_score,fifth_pref,fifth_score,match_type,new_names_y,loc_sort1
0,Uchcha Vighalay Jamhor Purwi Bhag.,1,Uchcha Vighalay Jamhor Purwi Bhag,100,Uchcha Vighalay Jamhor Purwi Madhya Bhag,90,Uchcha Vighalay Jamhor Pashchimi Bhag,86,Madhya Vighalay Jamhor Purwi Bhag,79,Uchcha Vighalay Jamhor Pashchimi Madhya Bhag,78,booth,Uchcha Vighalay Jamhor Purwi Bhag.,JAMHOR
1,Uchcha Vighalay Jamhor Purwi Madhya Bhag,2,Uchcha Vighalay Jamhor Purwi Madhya Bhag,100,Uchcha Vighalay Jamhor Purwi Bhag,90,Madhya Vighalay Jamhor Purwi Bhag,90,Uchcha Vighalay Jamhor Pashchimi Madhya Bhag,88,Madhya Vighalay Jamhor Pashchimi Bhag,83,booth,Uchcha Vighalay Jamhor Purwi Madhya Bhag,JAMHOR
2,Uchcha Vighalay Jamhor Pashchimi Madhya Bhag,3,Uchcha Vighalay Jamhor Pashchimi Madhya Bhag,100,Uchcha Vighalay Jamhor Pashchimi Bhag,91,Madhya Vighalay Jamhor Pashchimi Bhag,91,Uchcha Vighalay Jamhor Purwi Madhya Bhag,88,Uchcha Vighalay Jamhor Purwi Bhag,78,booth,Uchcha Vighalay Jamhor Pashchimi Madhya Bhag,JAMHOR
3,Uchcha Vighalay Jamhor Pashchimi Madhya Bhag,4,Uchcha Vighalay Jamhor Pashchimi Madhya Bhag,100,Uchcha Vighalay Jamhor Pashchimi Bhag,91,Madhya Vighalay Jamhor Pashchimi Bhag,91,Uchcha Vighalay Jamhor Purwi Madhya Bhag,88,Uchcha Vighalay Jamhor Purwi Bhag,78,booth,Uchcha Vighalay Jamhor Pashchimi Madhya Bhag,JAMHOR
4,Madhya Vighalay Jamhor Purwi Bhag,5,Madhya Vighalay Jamhor Purwi Bhag,100,Uchcha Vighalay Jamhor Purwi Madhya Bhag,90,Madhya Vighalay Jamhor Pashchimi Bhag,86,Uchcha Vighalay Jamhor Purwi Bhag,79,Uchcha Vighalay Jamhor Pashchimi Madhya Bhag,78,booth,Madhya Vighalay Jamhor Purwi Bhag,JAMHOR
5,Madhya Vighalay Jamhor Pashchimi Bhag,7,Madhya Vighalay Jamhor Pashchimi Bhag,100,Uchcha Vighalay Jamhor Pashchimi Madhya Bhag,91,Madhya Vighalay Jamhor Purwi Bhag,86,Uchcha Vighalay Jamhor Purwi Madhya Bhag,83,Uchcha Vighalay Jamhor Pashchimi Bhag,81,booth,Madhya Vighalay Jamhor Pashchimi Bhag,JAMHOR
6,Prathamik Vighalay Jokahari,8,Prathamik Vighalay Jokahari,100,Prathamik Vidyalay Khairi,85,Prathamik Vidhalay Pokharaha,84,Prathamik Vidyalay Gamhari,83,Prathamik Vidyalay Khaira,81,booth,Prathamik Vighalay Jokahari,JOKAHARI
7,Samudayik Bhavan Patanava,15,Samudayik Bhavan Patanava,100,Samudayik Bhavan Padarava,92,Samudayik Bhavan Paharama,88,Samudayik Bhavan Parasi,83,Samudayik Bhavan Parasadih,82,booth,Samudayik Bhavan Patanava,PATANAVA
8,Prathamik Vidyalay Devahara,16,Prathamik Vidyalay Devahara,100,Prathamik Vidyalay Devahara,100,Prathamik Vidyalay Devadhari,95,Prathamik Vidyalay Devariya,93,Prathamik Vidyalay Khaira,88,booth,Prathamik Vidyalay Devahara,DEVAHARA
9,Prathmik Vidyalay Jagadishapur,17,Prathamik Vidyalay Jagadishapur,98,Prathamik Vidyalay Basadiha,84,Prathamik Vidyalay Dariyapur,83,Prathamik Vidyalay Dhamarpur,83,Prathamik Vidyalay Gamhari,82,booth,Prathmik Vidyalay Jagadishapur,JAGAPUR


In [232]:
booth_match1.columns=['new_names','booth_num', 'first_pref', 'first_score', 'second_pref',\
       'second_score', 'third_pref', 'third_score', 'fourth_pref',\
       'fourth_score', 'fifth_pref', 'fifth_score', 'match_type',\
       'new_names_y', 'loc_sort1']

In [233]:
local_fin=matched.loc[:,['new_names','booth_num','loc_sort1','first_pref',\
       'first_score', 'second_pref', 'second_score', 'third_pref',\
       'third_score', 'fourth_pref', 'fourth_score', 'fifth_pref',\
       'fifth_score','match_type']]

booth_fin=booth_match1.loc[:,['new_names','booth_num', 'first_pref', 'first_score', 'second_pref',\
       'second_score', 'third_pref', 'third_score', 'fourth_pref',\
       'fourth_score', 'fifth_pref', 'fifth_score', 'match_type','loc_sort1']]




In [236]:
local_fin.shape,booth_fin.shape

((182, 14), (140, 14))

In [244]:
final_df=pd.concat([local_fin,booth_fin])

C:\anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [245]:
final_df.shape

(322, 14)

In [246]:
final_df.columns

Index(['booth_num', 'fifth_pref', 'fifth_score', 'first_pref', 'first_score',
       'fourth_pref', 'fourth_score', 'loc_sort1', 'match_type', 'new_names',
       'second_pref', 'second_score', 'third_pref', 'third_score'],
      dtype='object')

In [247]:
cols=['new_names','booth_num','loc_sort1','first_pref','first_score','second_pref','second_score',\
      'third_pref','third_score','fourth_pref','fourth_score','fifth_pref','fifth_score','match_type']
final_df=final_df[cols]

In [249]:
final_df.to_excel(r'final_mapped.xlsx',index=False)